In [ ]:
%pip install datasets transformers[sentencepiece]
# %pip install tensorflow
%pip install evaluate
%pip install scikit-learn
%pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 4.9 MB/s eta 0:00:000:00:0136m0:00:01:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam

import evaluate
from evaluate import evaluator

import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import Dataset, DatasetDict

/home/hugohdezm/miniconda3/envs/tf_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-05 15:38:29.121937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736091509.130958    6200 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736091509.133514    6200 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 15:38:29.142441: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performanc

In [ ]:
dataset = pd.read_csv('out2.csv')
dataset

,text,labels
0,my favourit food is anyth i didn t have to coo...,4
1,now if he doe off himself everyon will think h...,4
2,whi the fuck is bayless iso,0
3,to make her feel threaten,2
4,dirti southern wanker,0
...,...,...
44518,thank i wa diagnos with bp <NUMBER1> after t...,3
44519,well that make sens,3
44520,daddi issu name,4
44521,so glad i discov that subreddit a coupl month ...,3


In [ ]:
dataset.dtypes

text      object
labels     int64
dtype: object

In [ ]:
train_data, test_data = train_test_split(dataset, test_size=0.1, stratify=dataset['labels'], random_state=42)

# Subdividimos train en el propio train y datos para la validación
train_data, validation_data = train_test_split(train_data, test_size=0.2, stratify=train_data['labels'], random_state=3)

In [ ]:
print(train_data.head())
print(validation_data.head())
print(test_data.head())

                                                    text  labels
27409  we ve allway a ear to rent you here in reddit ...       3
13991  no you are just suggest someon be at the perfe...       4
7351   :face_with_tears_of_joy <> face_with_tears_of_...       3
21022           you first make it more interest than thi       6
5217               i guess you’v becom someth of an icon       4
                                                    text  labels
28161                                 definit bitter lol       3
21371                                      i just see ye       3
8280   i just don t see how thi movi wouldn t get dul...       0
19138             get your nose hair wax trust me on thi       3
16476      how come you shock the spinach in an ice bath       6
                                                    text  labels
4999   dear down voter go to your own sub and take yo...       0
33829  he s shoot veri effici too most of hi fg score...       3
5526   i feel like i m in

In [ ]:
train_dataset = Dataset.from_pandas(train_data)
validation_dataset = Dataset.from_pandas(validation_data)
test_dataset = Dataset.from_pandas(test_data)

huggingface_dataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

huggingface_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 32056
    })
    validation: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 8014
    })
    test: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 4453
    })
})

In [ ]:
huggingface_dataset['train'][6]

{'text': 'she must hate her father', 'labels': 0, '__index_level_0__': 29543}

In [ ]:
huggingface_dataset['train'].features

{'text': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
## Cambiar el checkpoint por otros (Cambio de tokenizador)
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

tokenized_datasets = huggingface_dataset.map(tokenize_function, batched=True)
tokenized_datasets.column_names

Map: 100%|██████████| 4453/4453 [00:00<00:00, 11397.25 examples/s]


{'train': ['text',
  'labels',
  '__index_level_0__',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'validation': ['text',
  'labels',
  '__index_level_0__',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'test': ['text',
  'labels',
  '__index_level_0__',
  'input_ids',
  'token_type_ids',
  'attention_mask']}

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['text', '__index_level_0__'])
tokenized_datasets = tokenized_datasets.with_format('tensorflow')
tokenized_datasets['train']

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 32056
})

In [ ]:
train_dataset = tokenized_datasets['train'].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'token_type_ids'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=8)

validation_dataset = tokenized_datasets['validation'].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'token_type_ids'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=8)

/home/hugohdezm/miniconda3/envs/tf_gpu/lib/python3.10/site-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
I0000 00:00:1736091559.461571    6200 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13892 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
# Cambie el checkpoint por otros (Cambio de modelo)
checkpoint = 'bert-base-cased'
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=7)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')

def compute_metrics(p):
  predictions, labels = p
  preds = np.argmax(predictions, axis=1)

  accuracy = accuracy_metric.compute(predictions=preds, references=labels)
  f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
  precision = precision_metric.compute(predictions=preds, references=labels, average="weighted")
  recall = recall_metric.compute(predictions=preds, references=labels, average="weighted")

  return {
      'accuracy': accuracy['accuracy'],
      'f1': f1['f1'],
      'precision': precision['precision'],
      'recall': recall['recall']
  }

In [ ]:
num_epochs = 3
num_train_steps = len(train_dataset) * num_epochs

import transformers
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0,
    decay_steps=num_train_steps
)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
# tf.config.set_visible_devices([], 'GPU')

history = model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=num_epochs,
  steps_per_epoch=len(train_dataset),
  validation_steps=len(validation_dataset),
)

Epoch 1/3
4007/4007 [==============================] - 2264s 563ms/step - loss: 1.1562 - accuracy: 0.5587 - val_loss: 1.4907 - val_accuracy: 0.3836
Epoch 2/3
4007/4007 [==============================] - 2175s 543ms/step - loss: 1.4982 - accuracy: 0.3687 - val_loss: 1.4880 - val_accuracy: 0.3836
Epoch 3/3
4007/4007 [==============================] - 2235s 558ms/step - loss: 1.4997 - accuracy: 0.3666 - val_loss: 1.4931 - val_accuracy: 0.3241


In [ ]:
dataset_name = 'feelnet-BERT'
saved_model_path = './{}_huggingface'.format(dataset_name.replace('/', '_'))

model.save(saved_model_path, include_optimizer=True)

INFO:tensorflow:Assets written to: ./feelnet-BERT_huggingface/assets


INFO:tensorflow:Assets written to: ./feelnet-BERT_huggingface/assets


In [ ]:
model.save_pretrained('./finetuned_model')
tokenizer.save_pretrained('./finetuned_model')

('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.txt',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [ ]:
## Recuperar el modelo guardado con save_pretrained
reloaded_tokenizer = AutoTokenizer.from_pretrained('./finetuned_model')
reloaded_model = TFAutoModelForSequenceClassification.from_pretrained('./finetuned_model')

Some layers from the model checkpoint at ./finetuned_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
